# Basic RoBERTa Fine-tuning Example

In this notebook, we will:

* Train a RoBERTa base model on MRPC and evaluate its performance

## Setup

#### Install dependencies

First, we will install libraries we need for this code.

In [ ]:
%%capture
!git clone https://github.com/jiant-dev/jiant.git
!cd jiant
!git checkout d26c213c742d36f8909f3a910694c8a90da416f1
!cd ..

In [ ]:
%%capture
# This Colab notebook already has its CUDA-runtime compatible versions of torch and torchvision installed
!pip install -r jiant/requirements-no-torch.txt
# Install pyarrow for nlp
!pip install pyarrow==0.16.0

#### Download data

Next, we will download task data. 

In [ ]:
%%capture
# Download MRPC data
!PYTHONPATH=/content/jiant python jiant/jiant/scripts/download_data/runscript.py \
    download \
    --tasks mrpc \
    --output_path=/content/tasks/

## `jiant` Pipeline

In [ ]:
import sys
sys.path.insert(0, "/content/jiant")

In [ ]:
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.proj.main.runscript as main_runscript
import jiant.shared.caching as caching
import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

#### Download model

Next, we will download a `roberta-base` model. This also includes the tokenizer.

In [ ]:
export_model.lookup_and_export_model(
    model_type="roberta-base",
    output_base_path="./models/roberta-base",
)

#### Tokenize and cache

With the model and data ready, we can now tokenize and cache the inputs features for our task. This converts the input examples to tokenized features ready to be consumed by the model, and saved them to disk in chunks.

In [ ]:
# Tokenize and cache each task
task_name = "mrpc"

tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"./tasks/configs/{task_name}_config.json",
    model_type="roberta-base",
    model_tokenizer_path="./models/roberta-base/tokenizer",
    output_dir=f"./cache/{task_name}",
    phases=["train", "val"],
))

We can inspect the first examples of the first chunk of each task.

In [ ]:
row = caching.ChunkedFilesDataCache("./cache/mrpc/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)

#### Writing a run config

Here we are going to write what we call a `jiant_task_container_config`. This configuration file basically defines a lot of the subtleties of our training pipeline, such as what tasks we will train on, do evaluation on, batch size for each task. The new version of `jiant` leans heavily toward explicitly specifying everything, for the purpose of inspectability and leaving minimal surprises for the user, even as the cost of being more verbose.

We use a helper "Configurator" to write out a `jiant_task_container_config`, since most of our setup is pretty standard. 

**Depending on what GPU your Colab session is assigned to, you may need to lower the train batch size.**

In [ ]:
jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="./tasks/configs",
    task_cache_base_path="./cache",
    train_task_name_list=["mrpc"],
    val_task_name_list=["mrpc"],
    train_batch_size=8,
    eval_batch_size=16,
    epochs=3,
    num_gpus=1,
).create_config()
os.makedirs("./run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, "./run_configs/mrpc_run_config.json")
display.show_json(jiant_run_config)

To briefly go over the major components of the `jiant_task_container_config`:

* `task_config_path_dict`: The paths to the task config files we wrote above.
* `task_cache_config_dict`: The paths to the task features caches we generated above.
* `sampler_config`: Determines how to sample from different tasks during training.
* `global_train_config`: The number of total steps and warmup steps during training.
* `task_specific_configs_dict`: Task-specific arguments for each task, such as training batch size and gradient accumulation steps.
* `taskmodels_config`: Task-model specific arguments for each task-model, including what tasks use which model.
* `metric_aggregator_config`: Determines how to weight/aggregate the metrics across multiple tasks.

#### Start training

Finally, we can start our training run. 

Before starting training, the script also prints out the list of parameters in our model.

In [ ]:
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/mrpc_run_config.json",
    output_dir="./runs/mrpc",
    model_type="roberta-base",
    model_path="./models/roberta-base/model/roberta-base.p",
    model_config_path="./models/roberta-base/model/roberta-base.json",
    model_tokenizer_path="./models/roberta-base/tokenizer",
    learning_rate=1e-5,
    eval_every_steps=500,
    do_train=True,
    do_val=True,
    do_save=True,
    force_overwrite=True,
)
main_runscript.run_loop(run_args)

At the end, you should see the evaluation scores for MRPC.